## Import dependencies

In [1]:
%cd ..

/mnt/c/Users/ankit/Desktop/Portfolio/kaggle/drawing-with-llms


In [2]:
import os
import random
import time
from typing import List

import pandas as pd
from camel.agents import ChatAgent
from camel.models.model_factory import ModelFactory, ModelPlatformType, ModelType
from datasets import load_dataset
from dotenv import load_dotenv
from pydantic import BaseModel
from tqdm import tqdm


load_dotenv(".env")

/home/ankith/anaconda3/envs/auto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## View Descriptions

In [3]:
model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type=ModelType.GEMINI_2_5_FLASH_PREVIEW,
    url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=os.getenv("GEMINI_API_KEY"),
)

In [4]:
data = load_dataset("csv", data_files="data/combined_train.csv")["train"]

print(data["description"])

['a purple forest at dusk', 'gray wool coat with a faux fur collar', 'a lighthouse overlooking the ocean', 'burgundy corduroy pants with patch pockets and silver buttons', 'orange corduroy overalls', 'a purple silk scarf with tassel trim', 'a green lagoon under a cloudy sky', 'crimson rectangles forming a chaotic grid', 'purple pyramids spiraling around a bronze cone', 'magenta trapezoids layered on a transluscent silver sheet', 'a snowy plain', 'black and white checkered pants', 'a starlit night over snow-covered peaks', 'khaki triangles and azure crescents', 'a maroon dodecahedron interwoven with teal threads']


We need more descriptions in order to consider Fine Tuning

Define a loop to randomly sample descriptions from the dataset and generate more similar samples using Gemini 2.5 Flash

In [5]:
# modified prompt from https://github.com/camel-ai/camel/blob/a48b948cbddd80b1cf5ad4d7ef99b3bfc21ec9ec/camel/datasets/few_shot_generator.py#L32


class SampleResponse(BaseModel):
    description: str
    rationale: str


class SampleList(BaseModel):
    items: List[SampleResponse]


def create_synthetic_dataset(num=100, num_gen=5):
    SYSTEM_PROMPT = """**You are an advanced data generation assistant.**  
    Your goal is to generate high-quality synthetic data samples based on 
    provided examples. Your output must be well-structured, 
    logically sound, and formatted correctly. 

    **Instructions:**
    1. **Follow the Structure**  
    Each data sample must include:  
    - **Description**: A clear, well-formed description of an SVG image.  
    - **Rationale**: A step-by-step, reasoning process for generation of the Description.  

    2. **Output Format (Strict)**
    - A list of json consisteing of five data samples
    ```
    [{"description" : "Generated description"
    "rationale": "reasoning why this description makes sense as a data sample"},
    ...
    ]
    ```

    **Now, generate 5 new data samples based on the given examples.**
    """

    total_samples = num
    num_samples_gen = num_gen
    sample_collection = []

    agent = ChatAgent(SYSTEM_PROMPT, model=model)

    for i in tqdm(range(int(total_samples / num_samples_gen))):
        num_examples = random.randrange(5, 10)
        indices = [random.randrange(0, len(data)) for _ in range(num_examples)]

        # every iteration randomly add samples from the dataset in context
        USER_PROMPT = (
            "<examples>" + "\n".join(data[indices]["description"]) + "</examples>"
        )

        sample_list = agent.step(USER_PROMPT, response_format=SampleList).msgs[0].parsed

        for sample in sample_list.items:
            sample_collection.append(sample.description)

        # reset user prompt
        agent.reset()
        # manual delay for rate limit
        time.sleep(2)

    if sample_collection:
        unique_samples = set(sample_collection)
        df = pd.DataFrame({"description": list(unique_samples)})
        df.dropna(inplace=True)
        return df


df_save = create_synthetic_dataset(5, 5)
# df_save.to_csv("data/descriptions.csv", index=False)
df_save.head()

100%|██████████| 1/1 [00:05<00:00,  5.17s/it]


,description
0,cyan triangles arranged on a black velvet surface
1,a blue desert under an orange sky
2,a red silk scarf with silver sequined fringe
3,a brown river under a dark blue starry sky
4,a faceted golden matte sphere
